# TABLE OF CONTENTS:
---
* [Notebook Summary](#Notebook-Summary)
* [Setup](#Setup)
    * [Notebook Parameters](#Notebook-Parameters)
    * [Connect to Workspace](#Connect-to-Workspace)
* [Data](#Data)
    * [Load Data](#Load-Data)
* [Model](#Model)
    * [Retrieve Model](#Retrieve-Model)
    * [Download Model](#Download-Model)
    * [Load Model](#Load-Model)
    * [Evaluate Model](#Evaluate-Model)
---

# Notebook Summary

This notebook will evaluate the model trained in `02_model_training` on the stanford dog test set and add the the calculated test accuracy to the properties of the registered Azure Machine Learning (AML) model.

# Setup

Append parent directory to sys path to be able to import created modules from src directory.

In [1]:
import sys
sys.path.append(os.path.dirname(os.path.abspath("")))

Automatically reload modules when changes are made.

In [2]:
%load_ext autoreload
%autoreload 2

Import libraries and modules.

In [3]:
# Import libraries
import azureml.core
import numpy as np
import os
import torch
from azureml.core import Model, Workspace

# Import created modules
from src.utils import load_data

print(f"azureml.core version: {azureml.core.VERSION}")

azureml.core version: 1.20.0


### Notebook Parameters

Specify the notebook parameters which are used in the source code below.

In [4]:
# Specify data source directory
data_src_dir = "../data"

# Specify the model name
model_name = "dog_clf_model"

# Specify the model version
model_version = None # None means latest

# Specify the directory path for outputs
outputs_dir = "../outputs"

# Specify the local model path
local_model_path = outputs_dir + "/dog_clf_model.pt"

### Connect to Workspace

In order to connect and communicate with the AML workspace, a workspace object needs to be instantiated using the AML SDK.

In [5]:
# Connect to the AML workspace using interactive authentication
ws = Workspace.from_config()

# Data

### Load Data

Use the previously created utility function to create dataloaders and retrieve dataset_sizes.

In [6]:
# Load data
dataloaders, dataset_sizes, class_names = load_data(data_src_dir)

# Model

### Retrieve Model

In [7]:
aml_model = Model(ws, name=model_name, version=model_version)

### Download Model

Download the model from the AML workspace to the local compute target.

In [8]:
# Create directory
outputs_folder = os.path.join(os.getcwd(), outputs_dir)
os.makedirs(outputs_folder, exist_ok=True)
print(f"Outputs folder {outputs_folder} has been created.")

# Download model artifact
aml_model.download(target_dir=outputs_dir, exist_ok=True)

Outputs folder /mnt/batch/tasks/shared/LS_root/mounts/clusters/mlopstemplatecibfdc24/code/Users/s.birk/pytorch-mlops-template-azure-ml/notebooks/../outputs has been created.


'../outputs/dog_clf_model.pt'

### Load Model

Load the downloaded model that was trained in the `02_model_training` notebook.

In [9]:
model = torch.load(local_model_path, map_location=lambda storage, loc: storage)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

### Evaluate Model

Evaluate the model on the test set by calculating the accuracy (which is our primary evaluation metric).

In [10]:
# Leverage GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize running_correct_preds (this will later be updated with every batch)
running_correct_preds = 0

# Use the test dataloader to load in data in batches
for inputs, labels in dataloaders["test"]:
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    # Make predictions on test data batch
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
    # Add correct predictions from current batch
    running_correct_preds += torch.sum(preds == labels.data)

In [11]:
# Calculate the total accuracy over the test set
test_accuracy = running_correct_preds.double() / dataset_sizes["test"]
print(f"Test accuracy: {np.round(test_accuracy.numpy(), 4)*100}%")

Test accuracy: 89.13%


Add the test accuracy to the model properties of the registered model.

In [12]:
try:
    aml_model.add_properties({"test_acc": str(np.round(test_accuracy.numpy(), 4)*100)})
except:
    print("Property could not be added.")

Model properties add operation complete.


Now you should be able to see your registered AML model with the test_acc property in the AML workspace:
    
<img src="../docs/images/aml_model_evaluated.png" alt="aml_model_evaluated" width="800"/>   